Day 12: Hill Climbing Algorithm ---
You try contacting the Elves using your handheld device, but the river you're following must be too low to get a decent signal.

You ask the device for a heightmap of the surrounding area (your puzzle input). The heightmap shows the local area from above broken into a grid; the elevation of each square of the grid is given by a single lowercase letter, where a is the lowest elevation, b is the next-lowest, and so on up to the highest elevation, z.

Also included on the heightmap are marks for your current position (S) and the location that should get the best signal (E). Your current position (S) has elevation a, and the location that should get the best signal (E) has elevation z.

You'd like to reach E, but to save energy, you should do it in as few steps as possible. During each step, you can move exactly one square up, down, left, or right. To avoid needing to get out your climbing gear, the elevation of the destination square can be at most one higher than the elevation of your current square; that is, if your current elevation is m, you could step to elevation n, but not to elevation o. (This also means that the elevation of the destination square can be much lower than the elevation of your current square.)

For example:

```
Sabqponm
abcryxxl
accszExk
acctuvwj
abdefghi
```

Here, you start in the top-left corner; your goal is near the middle. You could start by moving down or right, but eventually you'll need to head toward the e at the bottom. From there, you can spiral around to the goal:

```
v..v<<<<
>v.vv<<^
.>vv>E^^
..v>>>^^
..>>>>>^
```

In the above diagram, the symbols indicate whether the path exits each square moving up (^), down (v), left (<), or right (>). The location that should get the best signal is still E, and . marks unvisited squares.

This path reaches the goal in 31 steps, the fewest possible.

What is the fewest steps required to move from your current position to the location that should get the best signal?

In [8]:
class Vec2D:
    def __init__(self, x, y):
        self.tup = x, self.y = x, y

    def __str__(self):
        return '{:g}i + {:g}j'.format(self.tup[0], self.tup[1])

    def __repr__(self):
        return repr(self.tup)

    def __eq__(self, other):
        return self.tup[0] == other.tup[0] and self.tup[1] == other.tup[1]

    def __hash__(self):
        return hash(self.tup)

    def __sub__(self, other):
        return Vec2D(self.tup[0] - other.tup[0], self.tup[1] - other.tup[1])

    def __add__(self, other):
        return Vec2D(self.tup[0] + other.tup[0], self.tup[1] + other.tup[1])

    def __neg__(self):
        return Vec2D(-self.tup[0], -self.tup[1])

In [9]:
def find(grid, letter):
  width = len(grid[0])
  height = len(grid)

  for y in range(height):
    for x in range(width):
      if grid[y][x] == letter:
        return Vec2D(x, y)

In [4]:
import dotenv
import os
import requests

dotenv.load_dotenv()

input = requests.get(
  'https://adventofcode.com/2022/day/12/input',
  cookies={'session': os.getenv('ADVENT_OF_CODE_SESSION') }
)

In [26]:
grid = """\
Sabqponm
abcryxxl
accszExk
acctuvwj
abdefghi
""".splitlines()

grid = input.text.splitlines()

from https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm#Using_a_priority_queue

```
1  function Dijkstra(Graph, source):
2      dist[source] ← 0                           // Initialization
3
4      create vertex priority queue Q
5
6      for each vertex v in Graph.Vertices:
7          if v ≠ source
8              dist[v] ← INFINITY                 // Unknown distance from source to v
9              prev[v] ← UNDEFINED                // Predecessor of v
10
11         Q.add_with_priority(v, dist[v])
12
13
14     while Q is not empty:                      // The main loop
15         u ← Q.extract_min()                    // Remove and return best vertex
16         for each neighbor v of u:              // Go through all v neighbors of u
17             alt ← dist[u] + Graph.Edges(u, v)
18             if alt < dist[v]:
19                 dist[v] ← alt
20                 prev[v] ← u
21                 Q.decrease_priority(v, alt)
22
23     return dist, prev
```

In [27]:
# the built-in heapq has no way to decrease priorities
from heapdict import heapdict

def elevation(grid, pos):
  letter = grid[pos.tup[1]][pos.tup[0]]
  if letter == 'S':
    letter = 'a'
  elif letter == 'E':
    letter = 'z'
  return ord(letter) - ord('a')

def neighbors(grid, pos):
  width = len(grid[0])
  height = len(grid)

  for d in [Vec2D(1, 0), Vec2D(0, 1), Vec2D(-1, 0), Vec2D(0, -1)]:
    new_pos = pos + d
    if 0 <= new_pos.tup[0] < width and 0 <= new_pos.tup[1] < height and elevation(grid, new_pos) <= elevation(grid, pos) + 1:
      yield new_pos

def shortest_path(grid, start, end):
  width = len(grid[0])
  height = len(grid)

  dist = dict({ Vec2D(x, y): 1000000 for x in range(width) for y in range(height) })
  q = heapdict()
  dist[start] = 0
  for k in dist:
    q[k] = dist[k]

  while q:
    [u, distu] = q.popitem()
    if u == end:
      return distu

    for v in neighbors(grid, u):
      if not (v in q):
        continue
      alt = dist[u] + 1
      if alt < dist[v]:
        dist[v] = alt
        q[v] = alt

start = find(grid, 'S')
end = find(grid, 'E')

shortest_path(grid, start, end)

520

As you walk up the hill, you suspect that the Elves will want to turn this into a hiking trail. The beginning isn't very scenic, though; perhaps you can find a better starting point.

To maximize exercise while hiking, the trail should start as low as possible: elevation a. The goal is still the square marked E. However, the trail should still be direct, taking the fewest steps to reach its goal. So, you'll need to find the shortest path from any square at elevation a to the square marked E.

Again consider the example from above:

```
Sabqponm
abcryxxl
accszExk
acctuvwj
abdefghi
```

Now, there are six choices for starting position (five marked a, plus the square marked S that counts as being at elevation a). If you start at the bottom-left square, you can reach the goal most quickly:

```
...v<<<<
...vv<<^
...v>E^^
.>v>>>^^
>^>>>>>^
```

This path reaches the goal in only 29 steps, the fewest possible.

What is the fewest steps required to move starting from any square with elevation a to the location that should get the best signal?

In [28]:
width = len(grid[0])
height = len(grid)
starts = [Vec2D(x, y) for x in range(width) for y in range(height) if grid[y][x] == 'S' or grid[y][x] == 'a']
mins = [shortest_path(grid, start, end) for start in starts]
min(mins)

508